# Lecture 23 - Twitter Bot
In this notebook we will learn how to generate fake tweets using GPT-3.

<ol type = 1>
<li> Tweeting Fake Text</li>
    <ol type = a>
        <li> GPT-3</li>
    </ol>
<li> Perpetual Tweeting</li>  
<li> Monday Motivation Bot </li>
    
<li> Pacing and Leading Bot</li>

<li> Bot-Bot Conversation</li>


</ol>

    
    


# Clones, installs, and imports


## Clone GitHub Repository
This will clone the repository to your machine.  This includes the code and data files.  Then change into the directory of the repository.

In [1]:
!git clone https://github.com/zlisto/social_media_analytics

import os
os.chdir("social_media_analytics")

Cloning into 'social_media_analytics'...
remote: Enumerating objects: 441, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 441 (delta 2), reused 6 (delta 2), pack-reused 435
Receiving objects: 100% (441/441), 45.62 MiB | 17.41 MiB/s, done.
Resolving deltas: 100% (234/234), done.
Checking out files: 100% (55/55), done.


## Install Requirements 

In [2]:
!pip install -r requirements.txt --quiet


     |████████████████████████████████| 58 kB 3.0 MB/s 
     |████████████████████████████████| 4.9 MB 10.7 MB/s 
     |████████████████████████████████| 88 kB 7.6 MB/s 
     |████████████████████████████████| 1.6 MB 56.2 MB/s 
     |████████████████████████████████| 43 kB 1.8 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.8 MB 50.6 MB/s 
     |████████████████████████████████| 6.6 MB 43.6 MB/s 
     |████████████████████████████████| 163 kB 19.3 MB/s 
     |████████████████████████████████| 1.1 MB 66.0 MB/s 
     |████████████████████████████████| 163 kB 61.6 MB/s 
     |████████████████████████████████| 181 kB 63.7 MB/s 
     |████████████████████████████████| 162 kB 75.0 MB/s 
     |████████████████████████████████| 63 kB 1.2 MB/s 
     |████████████████████████████████| 162 kB 81.0 MB/s 
     |████████████████████████████████| 158 kB 65.9 MB/s 
     |██████

## API Keys

After cloning or pulling the repository, the file `scripts/config_bot.py` will be overwritten to a blank file.  You will need to edit the file by pasting in your OpenAI API key as follows: 

1. `OPENAI_API_KEY = 'your OpenAI API key`

We can use GPT-3 to generate text. First you need to create an account with OpenAI here: https://auth0.openai.com/u/signup?state=hKFo2SBWS3JUVEdmQmdzZXo5ckhpY3R5NEFlc2NPWWc3WHhvRqFur3VuaXZlcnNhbC1sb2dpbqN0aWTZIG9kTDB4LV83aEdnN3pRU3VUYnVZemlnZkFURFo2RDhno2NpZNkgRFJpdnNubTJNdTQyVDNLT3BxZHR3QjNOWXZpSFl6d0Q

Once you have an account, copy your API key from here: https://beta.openai.com/account/api-keys



## Import Packages

The important import is from `scripts.bot` where we have the bot command methods.

In [3]:
%load_ext autoreload
%autoreload 2

from scripts.api import *
from scripts.bot import *
from transformers import pipeline
import openai
import scripts.TextAnalysis as ta
import textwrap as tr

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import datetime
import requests
import json

pd.set_option("display.max_colwidth", None)


# Fake Tweets

We can use the GPT-3 transformer language model to generate fake tweets.

### GPT-3 Model



Set all the sampling parameters.  Choose the `input_text` to start the tweet.  You can write something like `"Write a positive tweet about Yale."`

In [5]:
input_text = "Write a positive tweet about Yale"
max_length = 1000
temperature = 1
p = 0.9
engine =  "text-davinci-002"

In [8]:
responses = openai.Completion.create(
  engine=engine,
  prompt= input_text,
  temperature=temperature,
  max_tokens=max_length,
  top_p=p,
  frequency_penalty=0,
  presence_penalty=0,
)

#print text
text = responses['choices'][0]['text']
print(f"GPT-3 Text:\n{tr.fill(text,width = 50)}")

GPT-3 Text:
    I'm so proud to be a part of the Yale
community! This university has such a rich history
and a bright future. Go Bulldogs!


# Perpetual Tweeting

We use a `while` loop combined with the `sleep` function to make the bot tweet perpetually.  The bot will tweet, then sleep for a random amount of time, continuously in a loop.  We can use a language model to create the tweets.

The mean sleep time is `tsleep_mean`, measured in seconds. We then add some noise to it to make it look more random to obtain the sleep time `tsleep`.  We also set `tweet_max` equal to the maximum number of tweets to generate.

## Perpetual tweeting with GPT-3

In [12]:
max_length = 1000
temperature = 1
p = 0.9

tsleep_mean = 1  #mean sleep time in seconds
tweet_max = 2 #maxium number of tweets to generate (it costs money to make GPT-3 write a tweet)
input_text = "Write a funny tweet about Yale being better than Harvard."

In [13]:
c = 0
while True:
    c+=1
    if c>tweet_max:break  #stop after tweet_max tweets
    responses = openai.Completion.create(
      engine=engine,
      prompt= input_text,
      temperature=temperature,
      max_tokens=max_length,
      top_p=p,
      frequency_penalty=0,
      presence_penalty=0,
    )

    #print text
    text = responses['choices'][0]['text']
    text = text.strip()  #remove leading and trailing white spaces

    print(f"{c}: {text}")
    
    tsleep = tsleep_mean + np.random.uniform(low=0.0, high=3)
    print(f"Sleeping for {tsleep:.2f} seconds\n")
    time.sleep(tsleep)

1: Ivy League schools are a joke. If you want a real education, go to Yale. Harvard is overrated.
Sleeping for 3.55 seconds

2: Yale is better than Harvard because our students are smarter, our campus is more beautiful, and our food is better.
Sleeping for 2.53 seconds



# Monday Motivation Bot

This bot will tweet something about #MondayMotivation each Monday.

In [14]:
max_length = 1000
temperature = 1
p = 0.9

tsleep_mean = 1  #mean sleep time in seconds
tweet_max = 2
input_text = "Write a clever tweet using the hashtag #MondayMotivation."

In [15]:
c = 0
while True:
    c+=1
    if c>tweet_max: break  #stop after tweet_max tweets
    responses = openai.Completion.create(
      engine=engine,
      prompt= input_text,
      temperature=temperature,
      max_tokens=max_length,
      top_p=p,
      frequency_penalty=0,
      presence_penalty=0,
    )

    #print text
    text = responses['choices'][0]['text']
    text = text.strip()
    
    print(f"{c}: {text}")
    
    
    tsleep = tsleep_mean + np.random.uniform(low=0.0, high=2)
    print(f"Sleeping for {tsleep:.2f} seconds\n")
    time.sleep(tsleep)

1: #MondayMotivation is a great day to get things done and check things off your to-do list!
Sleeping for 2.72 seconds

2: #MondayMotivation: Don't let the haters get you down.
Sleeping for 1.91 seconds



# Pacing and Leading Bot

This bot will implement a pacing and leading policy.  Set `sentiments` equal to the sentiment for each tweet as a list and set `topic` equal to the topic of your perusasion.  The sentiment should be a number between 0 and 5.

In [17]:
max_length = 1000
temperature = 1
p = 0.9

tsleep_mean = 1  #mean sleep time in seconds
topic = "Kanye West"
sentiments = list(range(0,6))

print(f"sentiments = {sentiments}")


sentiments = [0, 1, 2, 3, 4, 5]


In [18]:
print(f"Pacing and leading about {topic}")
c = 0
for sentiment in sentiments:
    c+=1
    input_text = f"Write a tweet with sentiment {sentiment}/5.0 about {topic}."

    responses = openai.Completion.create(
      engine=engine,
      prompt= input_text,
      temperature=temperature,
      max_tokens=max_length,
      top_p=p,
      frequency_penalty=0,
      presence_penalty=0,
    )

    #print text
    text = responses['choices'][0]['text']
    text = text.strip()
    
    print(f"Tweet {c}: sentiment = {sentiment}/5.0: {text}")
      
    
    tsleep = tsleep_mean + np.random.uniform(low=0.0, high=2)
    print(f"Sleeping for {tsleep:.2f} seconds\n")
    time.sleep(tsleep)

Pacing and leading about Kanye West
Tweet 1: sentiment = 0/5.0: Kanye West is the worst musician of all time.
Sleeping for 2.33 seconds

Tweet 2: sentiment = 1/5.0: Kanye is the worst. He's a washed up rapper who is only relevant because of his wife.
Sleeping for 1.18 seconds

Tweet 3: sentiment = 2/5.0: Kanye West is one of the most controversial figures in music today.
Sleeping for 2.51 seconds

Tweet 4: sentiment = 3/5.0: Kanye West is a controversial figure, but there's no denying his talent.
Sleeping for 1.02 seconds

Tweet 5: sentiment = 4/5.0: Kanye West is one of the most innovative and exciting artists of our generation.
Sleeping for 2.32 seconds

Tweet 6: sentiment = 5/5.0: Kanye West is a musical genius and one of the best rappers of all time.
Sleeping for 2.71 seconds



# Bot Conversation

We can make GPT-3 talk to itself.  Name `speaker0` and `speaker1` and describe them in  `desc0` and `desc1`.  Then set the first messages between them as `text0` and `text1`.  Set how many messages you want to generate with `msg_max`.  Then run the code and see what the spekaers have to say to each other.

In [19]:
msg_max = 6
speaker0 = "Goku"
speaker1 = "Vegeta"

context = f"The following is a conversation between {speaker0} and {speaker1}."

desc0 = f"{speaker0} is from Dragonball Z and is kind and funny."
desc1 = f"{speaker1} is from Dragonball Z and is angry and jealous."

text0 = f"{speaker0}: Hey Vegeta!"
text1 = f"{speaker1}: Go away Goku."

In [20]:
input_text = f"{context}\n{desc0}\n{desc1}\n\n{text0}\n{text1}"

print(input_text)

for i in range(msg_max):
    if i%2==0:
        speaker = speaker0
    elif i%2==1:
        speaker = speaker1

    input_text+=f"\n{speaker}:"
    responses = openai.Completion.create(
      engine=engine,
      prompt= input_text,
      temperature=temperature,
      max_tokens=max_length,
      top_p=p,
      frequency_penalty=0,
      presence_penalty=0.4,
    )   
    text = responses['choices'][0]['text']
    text = text.strip()
    #text +="\n"
    input_text += text
    print(f"{speaker}: {text}")
    time.sleep(2)


    
    


The following is a conversation between Goku and Vegeta.
Goku is from Dragonball Z and is kind and funny.
Vegeta is from Dragonball Z and is angry and jealous.

Goku: Hey Vegeta!
Vegeta: Go away Goku.
Goku: What's wrong?
Vegeta: I'm angry and jealous.
Vegeta: I'm angry because I can't beat you and I'm jealous because you're stronger than me.
Goku: Oh, I see.
Goku:Well, I guess I can understand that.
Vegeta:Shut up!
Vegeta: I don't want your sympathy!
Goku: Well, I'm sorry if you don't want my sympathy, but I do understand how you feel. It can be tough to not be the strongest and to always be second best.
Vegeta: Fine, whatever. Just leave me alone.
